# Graphs, functions and tracing 

In [1]:
import time
import tensorflow as tf

In [2]:
def inefficient_square(x):
    for i in tf.range(100):
        i ** 2
    return x ** 2

Simplifies, optimizes, paralellizes and prune the function.

In [3]:
tf.function(inefficient_square)

In [4]:
tf_inefficient_square = tf.function(inefficient_square)

In [5]:
tf_inefficient_square.python_function(2)

4

Each tf function generates a new graph for each signature it receives. (signature = shape + data type). We have to be careful with this. This is true for tensor input. If we pass python numbers, it will generate a new graph for each call.

In [6]:
%timeit -n10 -r10 inefficient_square(2)

50.9 ms ± 4.78 ms per loop (mean ± std. dev. of 10 runs, 10 loops each)


In [7]:
%timeit -n10 -r10 tf_inefficient_square(2)

The slowest run took 124.87 times longer than the fastest. This could mean that an intermediate result is being cached.
4.33 ms ± 11.3 ms per loop (mean ± std. dev. of 10 runs, 10 loops each)


In [8]:
two = tf.constant(2)

In [9]:
%timeit -n10 -r10 tf_inefficient_square(two)

The slowest run took 38.49 times longer than the fastest. This could mean that an intermediate result is being cached.
2.05 ms ± 4.53 ms per loop (mean ± std. dev. of 10 runs, 10 loops each)


In [10]:
print(tf.autograph.to_code(inefficient_square))

def tf__inefficient_square(x):
    with ag__.FunctionScope('inefficient_square', 'fscope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as fscope:
        do_return = False
        retval_ = ag__.UndefinedReturnValue()

        def get_state():
            return ()

        def set_state(block_vars):
            pass

        def loop_body(itr):
            i = itr
            (ag__.ld(i) ** 2)
        i = ag__.Undefined('i')
        ag__.for_stmt(ag__.converted_call(ag__.ld(tf).range, (100,), None, fscope), None, loop_body, get_state, set_state, (), {'iterate_names': 'i'})
        try:
            do_return = True
            retval_ = (ag__.ld(x) ** 2)
        except:
            do_return = False
            raise
        return fscope.ret(retval_, do_return)



In [11]:
@tf.function
def sum_squares(n):
    s = 0
    for i in tf.range(n + 1):
        s += i ** 2
    return s

In [12]:
print(tf.autograph.to_code(sum_squares.python_function))

def tf__sum_squares(n):
    with ag__.FunctionScope('sum_squares', 'fscope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as fscope:
        do_return = False
        retval_ = ag__.UndefinedReturnValue()
        s = 0

        def get_state():
            return (s,)

        def set_state(vars_):
            nonlocal s
            (s,) = vars_

        def loop_body(itr):
            nonlocal s
            i = itr
            s = ag__.ld(s)
            s += (i ** 2)
        i = ag__.Undefined('i')
        ag__.for_stmt(ag__.converted_call(ag__.ld(tf).range, ((ag__.ld(n) + 1),), None, fscope), None, loop_body, get_state, set_state, ('s',), {'iterate_names': 'i'})
        try:
            do_return = True
            retval_ = ag__.ld(s)
        except:
            do_return = False
            raise
        return fscope.ret(retval_, do_return)



- use all tf tool possible, including tf.range or tf.sort
- otherwise, they will only be evaluated at tracing time
- for example. if we use np.random to compute a randon number, it will be computed only when tracing the new graph
- the same if our function has side effects, like logging or printing
- it will only capture for loops that iterate through tensorlfow things
- use vectorized operations! don't use loops!

In [13]:
import numpy as np

In [14]:
@tf.function
def random_thing(input):
    return np.random.randint(0,10)

For each number, a new graph.

In [15]:
random_thing(1), random_thing(1), random_thing(2), random_thing(3)

(<tf.Tensor: shape=(), dtype=int32, numpy=7>,
 <tf.Tensor: shape=(), dtype=int32, numpy=7>,
 <tf.Tensor: shape=(), dtype=int32, numpy=3>,
 <tf.Tensor: shape=(), dtype=int32, numpy=0>)

For each new signature, a new graph

In [16]:
random_thing(tf.constant([1])), random_thing(tf.constant([2])), random_thing(tf.constant([3]))

(<tf.Tensor: shape=(), dtype=int32, numpy=6>,
 <tf.Tensor: shape=(), dtype=int32, numpy=6>,
 <tf.Tensor: shape=(), dtype=int32, numpy=6>)

In [17]:
random_thing(tf.constant([1.0, 2.0]))

<tf.Tensor: shape=(), dtype=int32, numpy=8>

In [18]:
@tf.function
def printer(input):
    print('Hi :). This means that the graph has been retraced!')

In [19]:
printer(1)

Hi :). This means that the graph has been retraced!


In [20]:
printer(1)

In [21]:
printer(2)

Hi :). This means that the graph has been retraced!


In [22]:
printer(tf.constant([1.0, 2.0]))

Hi :). This means that the graph has been retraced!


In [23]:
printer(tf.constant([3.0, 2.0]))

In [24]:
printer(tf.constant([1.0, 2.0, 3.0, 4.0]))

Hi :). This means that the graph has been retraced!


loops are only traced once

In [25]:
import time

@tf.function
def sq(x):
    for i in tf.range(10):
        i ** 2
        print(':)')
        time.sleep(2)
        print(':D')
    return x ** 2

In [26]:
sq(1)

:)
:D


<tf.Tensor: shape=(), dtype=int32, numpy=1>

In [27]:
sq(1)

<tf.Tensor: shape=(), dtype=int32, numpy=1>

## Graphs

python function -> tf function -> concrete function -> graph

Actually, for each signature, tensorflow generates a concrete function (a specific function for that signature) that generates the specific graph. And the executes that graph with the privoded input.

In [28]:
@tf.function
def sum_of_squares(x, y):
    return tf.add(tf.math.square(x), tf.math.square(y))

In [29]:
x = tf.constant([1.0, 1.0])
y = tf.constant([2.0, 2.0])
concrete_function = sum_of_squares.get_concrete_function(x, y)

This fuction generates the graph. Within that graph, tensors are translated into placeholders. Other things, like python numbers or numpy outputs, are constants in that graph.

We can get the graph, its operations an its placeholders.

In [30]:
concrete_function.graph

In [31]:
concrete_function.graph.get_operations()

[<tf.Operation 'x' type=Placeholder>,
 <tf.Operation 'y' type=Placeholder>,
 <tf.Operation 'Square' type=Square>,
 <tf.Operation 'Square_1' type=Square>,
 <tf.Operation 'Add' type=Add>,
 <tf.Operation 'Identity' type=Identity>]

And of course, the inputs and outputs of each of them, to get its connections:

(!) If repeated, operation names ends with `_` + number.

(!) Tensor names are created from the operations that generates that tensor. Their names olways end with `:` + number. 

In [32]:
concrete_function.graph.get_operations()[2].inputs

(<tf.Tensor 'x:0' shape=(2,) dtype=float32>,)

In [33]:
concrete_function.graph.get_operations()[2].outputs

[<tf.Tensor 'Square:0' shape=(2,) dtype=float32>]

We can give specific names to graph nodes (operations)

In [34]:
@tf.function
def sum_of_squares(x, y):
    return tf.add(x, y, name="ssum")

In [35]:
concrete_function = sum_of_squares.get_concrete_function(x, y)
concrete_function.graph.get_operations()

[<tf.Operation 'x' type=Placeholder>,
 <tf.Operation 'y' type=Placeholder>,
 <tf.Operation 'ssum' type=Add>,
 <tf.Operation 'Identity' type=Identity>]

We can get tensors and operations by name:

In [36]:
concrete_function.graph.get_operation_by_name('ssum')

<tf.Operation 'ssum' type=Add>

In [37]:
concrete_function.graph.get_tensor_by_name('ssum:0')

<tf.Tensor 'ssum:0' shape=(2,) dtype=float32>

And of course, we can get the signature of a function:

In [38]:
concrete_function.function_def.signature

name: "__inference_sum_of_squares_61449"
input_arg {
  name: "x"
  type: DT_FLOAT
}
input_arg {
  name: "y"
  type: DT_FLOAT
}
output_arg {
  name: "identity"
  type: DT_FLOAT
}

## Tracing details 

In [39]:
@tf.function
def sum_of_squares(x, y):
    print("x =", x)
    return tf.add(x, y, name="issum")

Look at this...

In [40]:
sum_of_squares(x, y)

x = Tensor("x:0", shape=(2,), dtype=float32)


<tf.Tensor: shape=(2,), dtype=float32, numpy=array([3., 3.], dtype=float32)>

x at tracing time is a tensor place holder! No info about the contents of the array at this moment. This is because tf first generates an optimized function, then a concrete function, after that, the graph is generated, and finally, the graph is feeded with the input tensor.

We can restrict the input signature if we want to prevent tensorflow from generate a of graphs.

In [41]:
@tf.function(input_signature=[tf.TensorSpec([2], tf.float32), tf.TensorSpec([2], tf.float32)])
def sum_of_squares(x, y):
    print("x =", x)
    return tf.add(x, y, name="issum")

In [42]:
sum_of_squares(x, y)

x = Tensor("x:0", shape=(2,), dtype=float32)


<tf.Tensor: shape=(2,), dtype=float32, numpy=array([3., 3.], dtype=float32)>

#### loops

Look a this:

In [43]:
@tf.function
def add_nodes(x):
    for i in range(5):
        print(i)
        x = tf.square(x)
    return x

In [44]:
add_nodes(1)

0
1
2
3
4


<tf.Tensor: shape=(), dtype=int32, numpy=1>

In [45]:
add_nodes.get_concrete_function(1).graph.get_operations()

[<tf.Operation 'Square/x' type=Const>,
 <tf.Operation 'Square' type=Square>,
 <tf.Operation 'Square_1' type=Square>,
 <tf.Operation 'Square_2' type=Square>,
 <tf.Operation 'Square_3' type=Square>,
 <tf.Operation 'Square_4' type=Square>,
 <tf.Operation 'Identity' type=Identity>]

In [46]:
@tf.function
def add_nodes(x):
    for i in tf.range(5):
        print(i)
        x = tf.square(x)
    return x

In [47]:
add_nodes(1)

Tensor("while/Placeholder:0", shape=(), dtype=int32)


<tf.Tensor: shape=(), dtype=int32, numpy=1>

Yes! If you use range, the tracing process will loop over the fuction n times, adding nodes. We have a loop that builds the graph. However, if we use tf.range, the loop will be included as part of the graph, and we'll have a graph with a loop inside, instead of a loop with n nodes.

#### Resources

Stateful objects (variables, queues, datasets ...) are resources in tensorflow. We can use them in out fuctions but there are a couple of improtant points:

* operations involving resources are executed in the order they appear. Stateless ones can be run in parallel, and the order is not guareanteed. (More info there)
* Resources are passed as reference to functions.

In [48]:
n = tf.Variable(1)

@tf.function
def sum1(n):
    return n.assign_add(1)

In [49]:
sum1.get_concrete_function(n).function_def.signature.input_arg[0]

name: "n"
type: DT_RESOURCE

Do not use =, +=, -= with tf variables! Use tf.assign, tf.assign_add, tf.assign_sub.